In [1]:
import numpy as np
import pandas as pd
import math
import re

In [2]:
# LUAD Lung adenocarcinoma 肺腺癌 
# 1表示横轴，方向从左到右；0表示纵轴，方向从上到下。
# 当axis=1时，如果是求平均，那么是从左到右横向求平均；
# 如果是拼接，那么也是左右横向拼接；如果是drop，那么也是横向发生变化，体现为列的减少。

In [3]:
# 字符串转16进制
def str2hex(s):
    odata = 0;
    su =s.upper()
    for c in su:
        tmp=ord(c)
        if tmp <= ord('9') :
            odata = odata << 4
            odata += tmp - ord('0')
        elif ord('A') <= tmp <= ord('F'):
            odata = odata << 4
            odata += tmp - ord('A') + 10
    return odata

In [4]:
# 文件预处理 processing

In [5]:
# 将数据分为tumor和normal组
def tumor_normal():
    # 原始数据读入
    express = pd.read_csv('E:/cancerPrognosis/MyLUAD/code/deal_download/express.txt', sep="\t")
    express = express.rename(columns={'Unnamed: 0':'gene_id'})
    # 删除gene_id列
    express.drop(labels=None, axis=1, index=None, columns='gene_id', level=None, inplace=True)
    # 获取人员的编号信息
    patient_id = list(express.columns[1:])
    # 划分癌症病人和正常病人以及转移组病人
    tumor = ['gene_name'] # 肿瘤病人id
    normal = ['gene_name'] # 正常病人id
    move = ['gene_name'] # 转移组病人id
    # 根据病人的id编号信息划分tumor和normal
    for i in patient_id:
        if i[13:16] == '11A':
            normal.append(i)
        elif i[13:16] == '01A':
            tumor.append(i)
        else:
            move.append(i)
    # 划分数据
    tumor_data = express[tumor]
    normal_data = express[normal]
    return tumor_data, normal_data

In [9]:
# 测试函数是使用
tumor_data, normal_data = tumor_normal()

In [6]:
# 整合相同基因名的数据，用均值代替
def merge_same_name_data():
    # 获取需要整合的数据
    tumor_data, normal_data = tumor_normal()
    print(tumor_data.shape)
    print(normal_data.shape)
    # 追踪基因整合过程
    j = 1
    # 新增加数据的起始位置
    row_in = tumor_data.shape[0]
    # 统计gene_name列数据出现的个数
    num = tumor_data['gene_name'].value_counts()
    print('一共有',len(num),'个基因')
    # 记录出现超过一次的基因名
    gene_names = []
    for i in range(len(num)):
        if num[i] > 1:
            gene_names.append(num[i:i+1].index[0])
    print('重复出现的基因有',len(gene_names),'个')
    # 开始合并重复出现的基因数据
    for gene in gene_names:
        # 记录重复出现的基因数据
        data_t = tumor_data[tumor_data['gene_name'] == gene]
        data_n = normal_data[normal_data['gene_name'] == gene]
        # 记录重复基因的索引
        row_index = list(data_t.index)
        # 删除重复基因
        tumor_data.drop(labels=None, axis=0, index=row_index, columns=None, level=None, inplace=True)
        normal_data.drop(labels=None, axis=0, index=row_index, columns=None, level=None, inplace=True)
        # 重新设置索引
        data_t.set_index('gene_name',inplace=True)
        data_n.set_index('gene_name',inplace=True)
        # 计算新的一行数据
        new_row_tumor = []
        new_row_normal = []
        # 为新数据添加基因名
        new_row_tumor.append(gene)
        new_row_normal.append(gene)
        # 计算新数据的基因表达量值
        data_tu = list(data_t.apply(lambda x: x.sum()))
        data_no = list(data_n.apply(lambda x: x.sum()))
        # 将新数据的基因名与基因表达量进行匹配
        new_row_tumor = new_row_tumor + data_tu
        new_row_normal = new_row_normal + data_no
        # 将新的一行数据添加如原始数据
        tumor_data.loc[row_in] = new_row_tumor
        normal_data.loc[row_in] = new_row_normal
        # 新的一行索引进行自增长
        row_in = row_in + 1
        if j%55 == 0:
            print('处理完成',j,'个基因')
            print(tumor_data.shape)
            print(normal_data.shape)
        j = j + 1
    return tumor_data, normal_data

In [13]:
# 测试函数时使用
tumor_data, normal_data = merge_same_name_data()
print(tumor_data.shape)
print(normal_data.shape)

(60660, 451)
(60660, 46)
一共有 59427 个基因
重复出现的基因有 110 个
处理完成 55 个基因
(59482, 451)
(59482, 46)
处理完成 110 个基因
(59427, 451)
(59427, 46)
(59427, 451)
(59427, 46)


In [7]:
# 初次筛选基因
# 删除在tumour与normal组中表达量均值之比在区间[0.8,1.2]的基因表达量数据
def compare_tumor_normal():
    tumor_data, normal_data = merge_same_name_data()
    # 计算tumor的均值tumor_mean
    tumor_data['tumor_mean'] = tumor_data.iloc[:,1:tumor_data.shape[1]].mean(axis=1)
    # 计算normal的均值normal_mean
    normal_data['normal_mean'] = normal_data.iloc[:,1:normal_data.shape[1]].mean(axis=1)
    # 计算比值列compare
    tumor_data['compare'] = (tumor_data['tumor_mean'] + 1) / (normal_data['normal_mean'] + 1)
    normal_data['compare'] = (tumor_data['tumor_mean'] + 1) / (normal_data['normal_mean'] + 1)
    # 获取均值比在[0.7,1.3]之间的数据索引
    tumor_da = tumor_data[tumor_data['compare'] > 0.7]
    normal_da = normal_data[normal_data['compare'] > 0.7]
    tumor_da = tumor_da[tumor_da['compare'] < 1.3]
    normal_da = normal_da[normal_da['compare'] < 1.3]
    tumor_da_index = tumor_da.index.to_list()
    normal_da_index = normal_da.index.to_list()
    # 剔除数据
    tumor_data.drop(labels=None, axis=0, index=tumor_da_index, columns=None, level=None, inplace=True)
    normal_data.drop(labels=None, axis=0, index=normal_da_index, columns=None, level=None, inplace=True)
    # 剔除多余的列
    # 剔除数据
    tumor_data.drop(labels=None, axis=1, index=None, columns=['tumor_mean','compare'], level=None, inplace=True)
    normal_data.drop(labels=None, axis=1, index=None, columns=['normal_mean','compare'], level=None, inplace=True)
    
    return tumor_data, normal_data

In [14]:
# 初次赛选基因函数调用
tumor_data, normal_data = compare_tumor_normal()
print(tumor_data.shape)
print(normal_data.shape)

(60660, 451)
(60660, 46)
一共有 59427 个基因
重复出现的基因有 110 个
处理完成 55 个基因
(59482, 451)
(59482, 46)
处理完成 110 个基因
(59427, 451)
(59427, 46)
(28585, 451)
(28585, 46)


In [8]:
# 合并tumor与normal数据用于limma数据差异分析
def normal_tumor():
    # 获取输入信息
    tumor_data, normal_data = compare_tumor_normal()
    # 设置第一列为索引
    normal_data.set_index('gene_name', inplace=True)
    tumor_data.set_index('gene_name', inplace=True)
    # 将两个文件进行合并
    normal_tumor = pd.concat([normal_data,tumor_data],axis=1,join='inner')
    # 对A列做log(x + 1)
    name = normal_tumor.columns.to_list()
    for na in name:
        normal_tumor[na] = normal_tumor[na].apply(np.log1p)
    # 存储文件
    normal_tumor.to_csv('E:/cancerPrognosis/MyLUAD/code/processing/normal_tumor.txt', sep='\t', header=True, index=True)
    return normal_tumor

In [15]:
# 函数调用，处理数据用于limma差异分析
normal_tumor = normal_tumor()
print(normal_tumor.shape)

(60660, 451)
(60660, 46)
一共有 59427 个基因
重复出现的基因有 110 个
处理完成 55 个基因
(59482, 451)
(59482, 46)
处理完成 110 个基因
(59427, 451)
(59427, 46)
(28585, 495)


In [16]:
# 将癌症组数据与临床数据整合
def clinical_express():
    # 获取express数据
    tumor_data, normal_data = compare_tumor_normal()
    # 设置索引
    tumor_data.set_index('gene_name', inplace=True)
    # 读取临床数据
    clinical = pd.read_csv('E:/cancerPrognosis/MyLUAD/code/deal_download/clinical.txt', sep="\t")
    # 获取样本名
    sample = clinical['sample'].to_list()
    # 根据临床数据删除已经剔除的样本信息
    del_sample = []
    exp_sample = tumor_data.columns.to_list()
    for i in exp_sample:
        if i[0:12] not in sample:
            del_sample.append(i)
    # 删除样本
    tumor_data.drop(labels=None, axis=1, index=None, columns=del_sample, level=None, inplace=True)
    # dataframe转置
    tumor_T = tumor_data.T
    # 去除重复出现的样本数据
    col_name = tumor_T.index.to_list()
    col_name1 = []
    for i in col_name:
        col_name1.append(i[0:12])
    tumor_T['name'] = col_name1
    # 获取重复数据的索引和个数
    values = tumor_T['name'].value_counts().to_list()
    indexs = tumor_T['name'].value_counts().index.to_list()
    del_sample_same = []
    for i in range(len(values)):
        if values[i] == 2:  
            index = tumor_T[tumor_T['name'] == indexs[i]].index.to_list()
            gap = str2hex(index[0][21:25]) - str2hex(index[1][21:25])
            if gap < 0:
                del_sample_same.append(index[1])
            else:
                del_sample_same.append(index[0])
        else:
            break
    # 删除重复数据
    tumor_T.drop(labels=None, axis=0, index=del_sample_same, columns=None, level=None, inplace=True)
    # 去除临床数据的冗余数据
    del_sample = []
    exp_sample = tumor_T.index.to_list()
    for i in exp_sample:
        del_sample.append(i[0:12])
    del_sample1 = []
    for i in sample:
        if i not in del_sample:
            del_sample1.append(i)
    # 设置索引
    clinical.set_index('sample', inplace=True)
    clinical.drop(labels=None, axis=0, index=del_sample1, columns=None, level=None, inplace=True)
    # 添加新的一列name作为合并条件
    tumor_T['sample'] = tumor_T.index.to_list()
    # 设置name列为索引
    tumor_T.set_index('name', inplace=True)
    # 将数据合并
    clinical_express = pd.concat([clinical,tumor_T],axis=1,join='inner')
    # 设置新的索引
    clinical_express.set_index('sample', inplace=True)
    # 存储数据
    clinical_express.to_csv('E:/cancerPrognosis/MyLUAD/code/processing/clinical_express.txt', sep='\t', header=True, index=True)
    return clinical_express

In [17]:
# 临床数据与基因表达数据整合函数调用
clinical_express = clinical_express()
print(clinical_express.shape)

(60660, 451)
(60660, 46)
一共有 59427 个基因
重复出现的基因有 110 个
处理完成 55 个基因
(59482, 451)
(59482, 46)
处理完成 110 个基因
(59427, 451)
(59427, 46)
(414, 28593)


In [18]:
# 根据limma差异分析的数据去除clinical_express中的多余数据
def limma_clinical_express():
    # 读取文件
    # 读取clinical_express数据
    cli_exp = pd.read_csv('E:/cancerPrognosis/MyLUAD/code/processing/clinical_express.txt', sep="\t")
    # 读取数据
    diff = pd.read_csv('E:/cancerPrognosis/MyLUAD/code/processing/limma/diff_RNA.txt', sep="\t")
    # 获取diff的基因名
    diff_name = diff.index.to_list()
    # 获取cli_exp的基因名
    cli_exp_name = cli_exp.columns.to_list()
    # 获取需要删除的基因名
    del_gene = []
    for name in cli_exp_name[9:]:
        if name not in diff_name:
            del_gene.append(name)
    # 删除基因数据
    cli_exp.drop(labels=None, axis=1, index=None, columns=del_gene, level=None, inplace=True)
    # 对A列做log(x + 1)
    name = cli_exp.columns.to_list()[9:]
    for na in name:
        cli_exp[na] = cli_exp[na].apply(np.log1p)
    # 更换基因名
    names = cli_exp.columns.to_list()
    new_names = []
    for i in names:
        new_names.append(re.sub('-','_',i))
    cli_exp.columns = new_names
    # 存储数据
    cli_exp.to_csv('E:/cancerPrognosis/MyLUAD/code/processing/coxMult/limma_clinical_express.txt', sep='\t', header=True, index=False)
    return cli_exp

In [19]:
# limma差异分析后基因删除数据函数调用
limma_clinical_express = limma_clinical_express()
print(limma_clinical_express.shape)

(414, 656)


In [20]:
limma_clinical_express.head()

,sample,time,status,gender,smoke,stage,T,M,N,AOC1,...,AC010998.3,AP000357.2,C13orf46,AC103718.1,AC067752.1,LCAL1,AC084871.3,AC008496.3,AL450345.2,DGCR5
0,TCGA-78-8655-01A-11R-2403-07,2360.0,0,0,1,I,T1,M0,N0,4.605170,...,0.000000,3.091042,3.433987,4.248495,0.000000,3.044522,1.609438,5.416100,0.000000,4.859812
1,TCGA-91-6829-01A-21R-1858-07,1258.0,1,1,1,I,T2,MX,N0,3.332205,...,0.693147,1.609438,1.945910,4.418841,0.000000,6.532334,1.945910,1.098612,0.693147,3.401197
2,TCGA-86-8672-01A-21R-2403-07,34.0,1,1,0,II,T3,M0,N0,3.583519,...,0.000000,0.000000,1.791759,6.284134,0.000000,5.062595,0.000000,1.098612,3.688879,4.043051
3,TCGA-62-8398-01A-11R-2326-07,444.0,1,1,1,III,T2,M0,N2,8.009363,...,0.693147,1.791759,4.094345,3.828641,0.693147,6.222576,1.609438,3.367296,4.962845,6.186209
4,TCGA-50-8459-01A-11R-2326-07,1119.0,0,1,0,II,T3,M0,N0,7.454720,...,2.397895,1.098612,4.762174,3.555348,0.693147,1.386294,3.951244,1.098612,1.791759,2.772589


In [21]:
# 根据通过Venn图取交集得到的基因名，剔除limma_clinical_express多余的数据
def limma_venn():
    # 读取文件
    # 读取clinical_express数据
    limma_cli_exp = pd.read_csv('E:/cancerPrognosis/MyLUAD/code/processing/coxMult/limma_clinical_express.txt', sep="\t")
    # 读取数据
    Venn = pd.read_csv('E:/cancerPrognosis/MyLUAD/code/processing/coxMult/VennData.txt', sep="\t")
    # 通过循环查找取交集的基因名
    CoxPH1 = Venn['CoxPH1'].to_list()
    CoxPH3 = Venn['CoxPH3'].to_list()
    CoxPH5 = Venn['CoxPH5'].to_list()
    CoxPH7 = Venn['CoxPH7'].to_list()
    CoxPH9 = Venn['CoxPH9'].to_list()
    # 取交集
    CoxPH = list(set(CoxPH1).intersection(set(CoxPH3)))
    CoxPH = list(set(CoxPH).intersection(set(CoxPH5)))
    CoxPH = list(set(CoxPH).intersection(set(CoxPH7)))
    CoxPH = list(set(CoxPH).intersection(set(CoxPH9)))
    # 获取数据列名
    cli_coln = limma_cli_exp.columns.to_list()[9:]
    # 获取需要删除的基因名
    del_coln = list(set(cli_coln).difference(set(CoxPH)))
    # 删除数据
    limma_cli_exp.drop(labels=None, axis=1, index=None, columns=del_coln, level=None, inplace=True)
    # 存储数据
    limma_cli_exp.to_csv('E:/cancerPrognosis/MyLUAD/code/processing/coxMult/venn_clinical_express.txt', sep='\t', header=True, index=False)
    return limma_cli_exp


In [22]:
# 韦恩图非交集数据的删除函数调用
venn_cli_exp = limma_venn()
venn_cli_exp.shape

(414, 88)

In [23]:
venn_cli_exp.head()

,sample,time,status,gender,smoke,stage,T,M,N,ANOS1,...,LINC00968,AC112206.2,AC046195.1,AC112777.1,AC026369.3,TUBB3,AC025580.1,LINC01833,AC135012.3,AP000357.2
0,TCGA-78-8655-01A-11R-2403-07,2360.0,0,0,1,I,T1,M0,N0,7.914983,...,3.218876,2.397895,2.772589,2.639057,3.713572,3.218876,3.178054,1.386294,1.791759,3.091042
1,TCGA-91-6829-01A-21R-1858-07,1258.0,1,1,1,I,T2,MX,N0,7.997999,...,3.465736,2.772589,0.000000,3.367296,0.693147,4.143135,6.030685,3.951244,0.693147,1.609438
2,TCGA-86-8672-01A-21R-2403-07,34.0,1,1,0,II,T3,M0,N0,6.109248,...,2.484907,0.693147,0.000000,2.833213,3.295837,4.025352,3.713572,3.044522,0.000000,0.000000
3,TCGA-62-8398-01A-11R-2326-07,444.0,1,1,1,III,T2,M0,N2,6.107023,...,2.397895,1.098612,1.386294,4.060443,1.386294,4.510860,4.762174,1.791759,0.000000,1.791759
4,TCGA-50-8459-01A-11R-2326-07,1119.0,0,1,0,II,T3,M0,N0,7.674153,...,3.761200,1.945910,0.000000,2.639057,4.174387,3.465736,4.025352,0.693147,1.791759,1.098612
